In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 2.x

In [ ]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y

!pip install tensorflow==2.2.0
!pip install keras==2.3.0
!pip install csbdeep
!pip install stardist
!pip install napari[all]

In [ ]:
%cd '/content/drive/My Drive/Yoloneat/'
!ls
import sys
import os
from glob import glob
sys.path.append("NEAT")
from NEATModels import NEATDynamic, nets
from NEATModels.config import dynamic_config
from NEATUtils import helpers
from NEATUtils.helpers import save_json, load_json

In [ ]:
npz_directory = '/content/drive/My Drive/CurieTrainingDatasets/Binning2V1data/'
npz_name = 'divisiondetectionbin2m4.npz'
npz_val_name = 'divisiondetectionbin2m4val.npz'

#Read and Write the h5 file, directory location and name
model_dir =  '/content/drive/My Drive/CurieDeepLearningModels/OneatModels/Binning2V1Models/'
model_name = 'bin2divisionmodeld47resf32.h5'

#Neural network parameters
division_categories_json = model_dir + 'DivisionCategories.json'
key_categories = load_json(division_categories_json)
division_cord_json = model_dir + 'DivisionCord.json'
key_cord = load_json(division_cord_json)

#For ORNET use residual = True and for OSNET use residual = False
residual = True
#NUmber of starting convolutional filters, is doubled down with increasing depth
startfilter = 32
#CNN network start layer, mid layers and lstm layer kernel size
start_kernel = 7
lstm_kernel = 3
mid_kernel = 3
#Network depth has to be 9n + 2, n= 3 or 4 is optimal for Notum dataset
depth = 47
#Size of the gradient descent length vector, start small and use callbacks to get smaller when reaching the minima
learning_rate = 1.0E-6
#For stochastic gradient decent, the batch size used for computing the gradients
batch_size = 2
# use softmax for single event per box, sigmoid for multi event per box
lstm_hidden_unit = 16
#Training epochs, longer the better with proper chosen learning rate
epochs = 250
nboxes = 1
#The inbuilt model stride which is equal to the nulber of times image was downsampled by the network
show = False

size_tminus = 4
size_tplus = 5
imagex = 64
imagey = 64
yolo_v0 = False
yolo_v1 = True
yolo_v2 = False


# In[4]:


config = dynamic_config(npz_directory =npz_directory, npz_name = npz_name, npz_val_name = npz_val_name, 
                         key_categories = key_categories, key_cord = key_cord, nboxes = nboxes, imagex = imagex,
                         imagey = imagey, size_tminus = size_tminus, size_tplus =size_tplus, epochs = epochs, yolo_v0 = yolo_v0, yolo_v1 = yolo_v1, yolo_v2 = yolo_v2,
                         residual = residual, depth = depth, start_kernel = start_kernel, mid_kernel = mid_kernel,
                         lstm_kernel = lstm_kernel, lstm_hidden_unit = lstm_hidden_unit, show = show,
                         startfilter = startfilter, batch_size = batch_size, model_name = model_name)

config_json = config.to_json()

print(config)
save_json(config_json, model_dir + os.path.splitext(model_name)[0] + '_Parameter.json')
# In[ ]:


Train = NEATDynamic(config, model_dir, model_name)

Train.loadData()

Train.TrainModel()

